## Library Imports and Setting up seeds

In [60]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
import tensorflow as tf
import torch.nn as nn
from math import sqrt
import pandas as pd
import numpy as np
import random
import torch
import os

# Setting up Seed for reproducibility
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# Using GPU if available
if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('Running on the GPU')
else:
  device = torch.device('cpu')
  print('Running on the CPU')

Running on the GPU


## Loading Dataset

In [61]:
# remove = ['P1', 'P11', 'P22', 'P25', 'P37', 'P4', 'P47', 'P48', 'P49', 'P54', 'P56', 'P59']
# remove.extend( ['P16', 'P24', 'P67', 'P68', 'P72','P74','P75','P30'] )

# Removing Samples where data had to be discarded due to some collection errors.
remove = ['P1', 'P11','P2', 'P22', 'P25', 'P37', 'P4', 'P47', 'P48', 'P49','P50', 'P54', 'P56', 'P59','P16', 'P24', 'P67', 'P68', 'P72','P74','P75','P30','P51','P58']

dataset = np.load('Autoclip_3000_5000_N95_FEATURES.npy',allow_pickle=True)
dataset = dataset[~np.isin(dataset[:,0],remove)]

X = pd.DataFrame(dataset[:,1:-2], dtype=float)
X = torch.tensor(X.values, dtype=torch.float32)

y_FEV1 = pd.DataFrame(dataset[:,-2], dtype=float)
y_FEV1 = torch.tensor(y_FEV1.values, dtype=torch.float32)

y_FVC = pd.DataFrame(dataset[:,-1], dtype=float)
y_FVC = torch.tensor(y_FVC.values, dtype=torch.float32)


print(X.shape,y_FEV1.shape,y_FVC.shape)

torch.Size([47, 710]) torch.Size([47, 1]) torch.Size([47, 1])


## Defining MLP Model

In [62]:
class MLP(nn.Module):
  def __init__(self,input_size=710,output_size=1,layers=[64,64,16]):
    super().__init__()

    # initialize the input layer
    self.linear1=nn.Linear(input_size,layers[0])
    torch.nn.init.uniform_(self.linear1.weight,-sqrt(6/input_size), sqrt(6/input_size))

    # initialize the hidden layers
    self.linear2=nn.Linear(layers[0],layers[1])
    torch.nn.init.uniform_(self.linear2.weight,-sqrt(6/layers[0]), sqrt(6/layers[0]))

    # self.linear3=nn.Linear(layers[1],layers[2])
    # torch.nn.init.uniform_(self.linear3.weight,-sqrt(6/layers[1]), sqrt(6/layers[1]))

    # self.linear4=nn.Linear(layers[2],layers[3])
    # torch.nn.init.uniform_(self.linear4.weight,-sqrt(6/layers[2]), sqrt(6/layers[2]))

    # # initialize the output layer
    self.linear5=nn.Linear(layers[1],layers[2])
    torch.nn.init.uniform_(self.linear5.weight,-sqrt(6/layers[1]), sqrt(6/layers[1]))

    self.linear_out=nn.Linear(layers[2],output_size)
    torch.nn.init.uniform_(self.linear_out.weight,-sqrt(6/layers[2]), sqrt(6/layers[2]))



  def forward(self,X):
    # X = nn.functional.relu(self.linear1(X))
    # X = nn.functional.relu(self.linear2(X))
    # X = nn.functional.relu(self.linear3(X))
    X = torch.sin(self.linear1(30*X))
    X = torch.sin(self.linear2(30*X))
    # X = torch.sin(self.linear3(50*X))
    # X = torch.sin(self.linear4(50*X))
    X = torch.sin(self.linear5(30*X))
    X = self.linear_out(X)
    return X

model = MLP()
print(model)
criterion = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

MLP(
  (linear1): Linear(in_features=710, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=64, bias=True)
  (linear5): Linear(in_features=64, out_features=16, bias=True)
  (linear_out): Linear(in_features=16, out_features=1, bias=True)
)


In [63]:
def LOOCV_MLP(MLP,X,y,num_epochs= 100):

    loo     = LeaveOneOut()
    prog    = 0
    tot     = len(X)
    y_GT    = []
    y_PT    = []

    for i,(train_index, test_index) in enumerate(loo.split(X)):

        prog = prog + 1
        print("Fold : {0}/{1}".format(prog,tot))

        X_Train, X_Test = X[train_index],X[test_index]
        y_Train, y_Test = y[train_index],y[test_index]

        model = MLP(input_size=X.shape[1]).to(device)
        criterion = nn.SmoothL1Loss()
        optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

        train_dataset = DataLoader(list(zip(X_Train,y_Train)), shuffle=True, batch_size=10)
        test_dataset = DataLoader(list(zip(X_Test,y_Test)), shuffle=True, batch_size=1)

        for epochs in range(num_epochs):
            for data,target in train_dataset:
                data,target = data.to(device),target.to(device)
                out = model(data)
                loss = criterion(out, target)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        for data,target in test_dataset:
                y_GT.append(target.cpu().detach().numpy().ravel())
                out = model(data.to(device))
                y_PT.append(out.cpu().detach().numpy().ravel())

    print("Training Completed \n")

    print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=")
    print("MLP metrics : ")
    print("\tMean Absolute Percentage Error : " , 100 * mean_absolute_percentage_error(y_GT,y_PT))
    print("\tMean Aabsolute Error : " , mean_absolute_error(y_GT,y_PT))
    print("\tMean Square Error : " , mean_squared_error(y_GT,y_PT,squared=False))
    print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=")

## Training MLP model for estimating FEV1

In [64]:
LOOCV_MLP(MLP,X,y_FEV1,num_epochs=100)

Fold : 1/47
Fold : 2/47
Fold : 3/47
Fold : 4/47
Fold : 5/47
Fold : 6/47
Fold : 7/47
Fold : 8/47
Fold : 9/47
Fold : 10/47
Fold : 11/47
Fold : 12/47
Fold : 13/47
Fold : 14/47
Fold : 15/47
Fold : 16/47
Fold : 17/47
Fold : 18/47
Fold : 19/47
Fold : 20/47
Fold : 21/47
Fold : 22/47
Fold : 23/47
Fold : 24/47
Fold : 25/47
Fold : 26/47
Fold : 27/47
Fold : 28/47
Fold : 29/47
Fold : 30/47
Fold : 31/47
Fold : 32/47
Fold : 33/47
Fold : 34/47
Fold : 35/47
Fold : 36/47
Fold : 37/47
Fold : 38/47
Fold : 39/47
Fold : 40/47
Fold : 41/47
Fold : 42/47
Fold : 43/47
Fold : 44/47
Fold : 45/47
Fold : 46/47
Fold : 47/47
Training Completed 

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
MLP metrics : 
	Mean Absolute Percentage Error :  5.840282887220383
	Mean Aabsolute Error :  0.18141241
	Mean Square Error :  0.24212278
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


## Training MLP model for estimating FVC

In [65]:
LOOCV_MLP(MLP,X,y_FVC,num_epochs=100)

Fold : 1/47
Fold : 2/47
Fold : 3/47
Fold : 4/47
Fold : 5/47
Fold : 6/47
Fold : 7/47
Fold : 8/47
Fold : 9/47
Fold : 10/47
Fold : 11/47
Fold : 12/47
Fold : 13/47
Fold : 14/47
Fold : 15/47
Fold : 16/47
Fold : 17/47
Fold : 18/47
Fold : 19/47
Fold : 20/47
Fold : 21/47
Fold : 22/47
Fold : 23/47
Fold : 24/47
Fold : 25/47
Fold : 26/47
Fold : 27/47
Fold : 28/47
Fold : 29/47
Fold : 30/47
Fold : 31/47
Fold : 32/47
Fold : 33/47
Fold : 34/47
Fold : 35/47
Fold : 36/47
Fold : 37/47
Fold : 38/47
Fold : 39/47
Fold : 40/47
Fold : 41/47
Fold : 42/47
Fold : 43/47
Fold : 44/47
Fold : 45/47
Fold : 46/47
Fold : 47/47
Training Completed 

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
MLP metrics : 
	Mean Absolute Percentage Error :  5.930667370557785
	Mean Aabsolute Error :  0.19943205
	Mean Square Error :  0.28832754
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
